In [1]:
!ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images
# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
# !ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images 
# !ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

In [ ]:
!python train.py --dataset CUB --train_aug --method baseline

tracking in baseline train: True
USE pre-trained model: False
Checkpoint path: ckpts/CUB/_resnet18_baseline_aug_tracking_lr0.0010
Fresh wandb run
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210603_040959-1t4z3gnk
wandb: Syncing run graceful-spaceship-29
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/1t4z3gnk
wandb: Run `wandb off` to turn off syncing.

